In [6]:
import numpy as np
from PIL import Image
import os

def create_segmentation_array(image_path):
    # Open the grayscale image
    img = Image.open(image_path).convert("L")

    # Convert the image to a NumPy array
    img_array = np.array(img)

    # Get unique values in the image
    unique_values = np.unique(img_array)

    # Create a dictionary to map unique values to class labels
    class_mapping = {value: index for index, value in enumerate(unique_values)}

    # Create a new array with class labels
    labeled_array = np.vectorize(class_mapping.get)(img_array)

    return labeled_array, unique_values

# Example usage
for filename in os.listdir("/Users/sanemleblebici/Downloads/Car parts segmentation/test"):
        if filename.endswith(".png"):
            image_path = os.path.join('/Users/sanemleblebici/Downloads/Car parts segmentation/test', filename)
            segmentation_array, unique_values = create_segmentation_array(image_path)
            
            # Print the unique values
            print("Unique Values in the Image:")
            print(unique_values)

            print("Unique Values in the Mask:")
            print(np.unique(segmentation_array))


Unique Values in the Image:
[ 0  2  3  4  9 18 28 29 30 32 33 36 38 39 41 42 44 45 46 47]
Unique Values in the Mask:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
Unique Values in the Image:
[ 0  2  3  6  9 10 13 14 22 27 28 29 32 34 38 41 44 46 47]
Unique Values in the Mask:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
Unique Values in the Image:
[ 0  3  6  9 10 13 14 22 27 28 32 34 38 41 42 44]
Unique Values in the Mask:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
Unique Values in the Image:
[ 0  2  3  9 18 26 29 30 32 33 41 42 44 45]
Unique Values in the Mask:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
Unique Values in the Image:
[ 0  3  6  9 10 14 16 17 20 21 22 23 27 34 38 39 41 42 44 47]
Unique Values in the Mask:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
Unique Values in the Image:
[ 0  6  9 10 14 16 17 19 20 21 22 27 33 38 41 44]
Unique Values in the Mask:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
Unique Values in

### MAKE NP ARRAYS WITH 9 CLASSES

In [15]:
import os
import numpy as np
from PIL import Image

# Define the classes you want to include with their corresponding pixel values
included_classes = {
    1: [14],                 # hood
    2: [20, 32],             # left_front_door, right_front_door
    3: [28, 16],             # left_back_door, right_back_door
    4: [44],                 # tire
    5: [23, 36],             # left_quater_glass, right_quarter_glass
    6: [45],                 # trunk
    7: [3],                  # back_fender
    8: [6]                   # bumper
}

def create_segmentation_array(image_path):
    # Open the grayscale image
    img = Image.open(image_path).convert("L")

    # Resize the image to 256x256
    img = img.resize((256, 256), Image.ANTIALIAS)

    # Convert the resized image to a NumPy array
    img_array = np.array(img)

    # Initialize the labeled array with zeros
    labeled_array = np.zeros_like(img_array, dtype=np.uint8)

    # Map each class to its corresponding pixel values
    for class_label, class_values in included_classes.items():
        labeled_array[np.isin(img_array, class_values)] = class_label

    # Set all other values as class 0 (background)
    labeled_array[labeled_array == 0] = 0

    return labeled_array

def process_images_in_directory(directory_path, output_directory):
    for filename in os.listdir(directory_path):
        if filename.endswith(".png"):
            # Remove "_mask" from the PNG file name
            filename_no_extension = os.path.splitext(filename)[0]
            if filename_no_extension.endswith("_mask"):
                filename_no_extension = filename_no_extension[:-5]

            image_path_png = os.path.join(directory_path, filename)

            # Create the segmentation array
            segmentation_array = create_segmentation_array(image_path_png)

            # Create the corresponding JPG file path
            image_path_jpg = os.path.join(directory_path, f"{filename_no_extension}.jpg")

            # Check if the corresponding JPG file exists
            if os.path.exists(image_path_jpg):
                # Open the JPG image and resize to 256x256
                img = Image.open(image_path_jpg).resize((256, 256), Image.ANTIALIAS)

                # Convert the resized image to a NumPy array (RGB)
                rgb_array = np.array(img)

                # Merge RGB array and segmentation array
                merged_array = np.dstack((rgb_array, segmentation_array))

                # Save the merged array to a .npy file
                output_path = os.path.join(output_directory, f"{filename_no_extension}.npy")
                np.save(output_path, merged_array)

# Replace "path/to/your/directory" with the actual path to your directory containing PNG files
# Replace "path/to/output/directory" with the directory where you want to save the .npy files
input_directory = "/Users/sanemleblebici/Downloads/Car parts segmentation/test"
output_directory = "/Users/sanemleblebici/Desktop/deep_learning_project/data/car_dataset/DATA"
process_images_in_directory(input_directory, output_directory)


/var/folders/mt/36j75y4n7xz6rfcxjxyyk60m0000gn/T/ipykernel_70428/1369323281.py:22: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((256, 256), Image.ANTIALIAS)
/var/folders/mt/36j75y4n7xz6rfcxjxyyk60m0000gn/T/ipykernel_70428/1369323281.py:58: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = Image.open(image_path_jpg).resize((256, 256), Image.ANTIALIAS)


### MOVE FILES TO IMAGES AND MASKS

In [10]:
import os
import shutil

def organize_files(input_directory, output_directory, file_extension):
    # Create output subdirectory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    # Iterate through files in the input directory
    for filename in os.listdir(input_directory):
        if filename.endswith(file_extension):
            print(filename)
            source_path = os.path.join(input_directory, filename)
            destination_path = os.path.join(output_directory, filename)
            
            try:
                shutil.copy(source_path, destination_path)
                print(f"Copied: {source_path} -> {destination_path}")
            except Exception as e:
                print(f"Error copying {source_path} to {destination_path}: {e}")

def organize_all_directories():
    main_directories = ['/Users/sanemleblebici/Desktop/Car/test', '/Users/sanemleblebici/Desktop/Car/valid', '/Users/sanemleblebici/Desktop/Car/train']
    png_output_directory = '/Users/sanemleblebici/Desktop/deep_learning_project/data/car_dataset/'
    jpg_output_directory = '/Users/sanemleblebici/Desktop/deep_learning_project/data/car_dataset/images'

    for main_directory in main_directories:
        organize_files(main_directory, png_output_directory, '.png')
        organize_files(main_directory, jpg_output_directory, '.jpg')

organize_all_directories()


9ea2bb2bfca3_jpg.rf.b87eebe45d4df20754319250f6309c71_mask.png
Copied: /Users/sanemleblebici/Desktop/Car/test/9ea2bb2bfca3_jpg.rf.b87eebe45d4df20754319250f6309c71_mask.png -> /Users/sanemleblebici/Desktop/deep_learning_project/data/car_dataset/9ea2bb2bfca3_jpg.rf.b87eebe45d4df20754319250f6309c71_mask.png
IMG_9215_jpg.rf.45c85ec6155639365c84a9eb2c9b8f54_mask.png
Copied: /Users/sanemleblebici/Desktop/Car/test/IMG_9215_jpg.rf.45c85ec6155639365c84a9eb2c9b8f54_mask.png -> /Users/sanemleblebici/Desktop/deep_learning_project/data/car_dataset/IMG_9215_jpg.rf.45c85ec6155639365c84a9eb2c9b8f54_mask.png
48e8289154fe_jpg.rf.466bbfc8d444d5d718919ada374905dd_mask.png
Copied: /Users/sanemleblebici/Desktop/Car/test/48e8289154fe_jpg.rf.466bbfc8d444d5d718919ada374905dd_mask.png -> /Users/sanemleblebici/Desktop/deep_learning_project/data/car_dataset/48e8289154fe_jpg.rf.466bbfc8d444d5d718919ada374905dd_mask.png
cade3111bbec_jpg.rf.192fcb3d3b1ecfa73d775cfef9b5d754_mask.png
Copied: /Users/sanemleblebici/Deskt

### MAKE FILENAMES NUMBERS

In [16]:
import os

def rename_files_to_sequential_numbers(directory):
    # Get the list of files in the directory
    files = os.listdir(directory)

    # Sort the files to ensure a consistent order
    files.sort()

    # Iterate through the files and rename them with sequential numbers
    for index, filename in enumerate(files, start=1):
        if filename.lower() == ".ds_store":
            continue  # Skip .DS_Store files

        original_path = os.path.join(directory, filename)
        new_filename = f"{index:04d}.npy"  # Using four-digit numbers (e.g., 0001, 0002, ...)
        new_path = os.path.join(directory, new_filename)

        # Rename the file
        os.rename(original_path, new_path)

        print(f"Renamed: {filename} -> {new_filename}")

# Example usage:
directory_to_rename = '/Users/sanemleblebici/Desktop/deep_learning_project/data/car_dataset/DATA'
rename_files_to_sequential_numbers(directory_to_rename)


Renamed: 02bb167bc516_jpg.rf.2bd2ee75a4d7597ce2373c76fd24b932.npy -> 0001.npy
Renamed: 02bb167bc516_jpg.rf.87b9c93ba6beffaeeca5589e454af096.npy -> 0002.npy
Renamed: 02bb167bc516_jpg.rf.c6a93ebe77afcde66afc28c5a64ca08d.npy -> 0003.npy
Renamed: 02cb7efd96bb_jpg.rf.6e6dd4e6650d79f2c6f1f752a184ba42.npy -> 0004.npy
Renamed: 052ab9853a44_jpg.rf.08bb19e3ce236b32dc6bc10be16ead4a.npy -> 0005.npy
Renamed: 05ad71b40835_jpg.rf.779aa9a5e69b56ab654d0063aeba27e1.npy -> 0006.npy
Renamed: 0907b29700b6_jpg.rf.34129ec9661a68940857c9f497013c4f.npy -> 0007.npy
Renamed: 0a5c68c2435f_jpg.rf.24ab39e1cb425e71ffdac3464f3f15d1.npy -> 0008.npy
Renamed: 0a5c68c2435f_jpg.rf.255157ee4103dd737afe400ad51e3745.npy -> 0009.npy
Renamed: 0a5c68c2435f_jpg.rf.f75af1f6d248f3505bf572ff79c1af63.npy -> 0010.npy
Renamed: 0a88116edc9b_jpg.rf.d89e6659564e09f789dd3c2bfa1210a9.npy -> 0011.npy
Renamed: 0c7fec6fbd12_jpg.rf.22ec00ac2225b97ae64a758a1f003d11.npy -> 0012.npy
Renamed: 0c7fec6fbd12_jpg.rf.78897e1df264c4dcaef7c7a98bbc0d7d.np

### RESIZE AND SAVE

In [ ]:
import os
import cv2

def resize_images_in_directory(directory, target_size=(256, 256)):
    # Iterate through files in the directory
    for filename in os.listdir(directory):
        # Check if the file is an image (you may need to add more file format checks)
        if filename.lower().endswith(('.jpg')):
            # Construct the full path to the image
            image_path = os.path.join(directory, filename)

            # Read the image
            image = cv2.imread(image_path)

            if image is not None:
                # Resize the image
                resized_image = cv2.resize(image, target_size)

                # Save the resized image, overwriting the original
                cv2.imwrite(image_path, resized_image)

                print(f"Resized and saved: {image_path}")
            else:
                print(f"Error loading the image: {image_path}")

# Example usage:
directory_to_resize = '/Users/sanemleblebici/Desktop/deep_learning_project/data/car_dataset/images'
resize_images_in_directory(directory_to_resize)

### MAKE GREYSCALE MASKS FROM .npy ARRAYS

In [17]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize

def save_array_to_image(array, output_path, cmap, title="Image"):
    """
    Convert a NumPy array to an image and save it.

    Parameters:
    - array (numpy.ndarray): 2D NumPy array representing the image.
    - output_path (str): Path to save the image.
    - cmap (str or None): Colormap for the image. If None, the image will be treated as an RGB image.
    - title (str): Title of the displayed image.
    """
    plt.imshow(array, cmap=cmap)
    plt.axis('off')

    # Save the image
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0.0)
    plt.close()

def process_npy_files(input_directory, output_directory_gray, output_directory_rgb):
    # Ensure the output directories exist
    if not os.path.exists(output_directory_gray):
        os.makedirs(output_directory_gray)
    if not os.path.exists(output_directory_rgb):
        os.makedirs(output_directory_rgb)

    # Iterate through .npy files in the input directory
    for filename in os.listdir(input_directory):
        if filename.lower().endswith('.npy'):
            input_path = os.path.join(input_directory, filename)

            # Load the NumPy array from the .npy file
            full_array = np.load(input_path)

            # Extract the 4th channel if the array is 4-dimensional
            array = full_array[:, :, 3]
                
            # Save the grayscale image with the same base filename but changed extension to .png
            output_path_gray = os.path.join(output_directory_gray, os.path.splitext(filename)[0])
            save_array_to_image(array, output_path_gray, cmap='gray', title="Digits Image")

            # Save the RGB image with the first three channels
            rgb_array = full_array[:, :, :3]
            output_path_rgb = os.path.join(output_directory_rgb, os.path.splitext(filename)[0] + '.jpg')
            save_array_to_image(rgb_array, output_path_rgb, cmap=None, title="RGB Image")

# Example usage:
input_directory = '/Users/sanemleblebici/Desktop/deep_learning_project/data/car_dataset/DATA'
output_directory_gray = '/Users/sanemleblebici/Desktop/deep_learning_project/data/car_dataset/masks'
output_directory_rgb = '/Users/sanemleblebici/Desktop/deep_learning_project/data/car_dataset/images'

process_npy_files(input_directory, output_directory_gray, output_directory_rgb)

